Import Library

In [2]:
from serpapi import GoogleSearch
from urllib.parse import *

In [3]:

params = {
  "engine": "google_maps",
  "q": "hotel", # objek yang dicari
  "ll": "@-7.791265,110.366071,14.84z", #koordinat tempat sekitar
  "type": "search",
  "api_key": "YOUR_API_KEY"
}

search = GoogleSearch(params)

lIdx = 0 # indeks yang ditampilkan dari data
lSum = 7 # Batas jumlah data yang ditampilkan
local_results = []

while lIdx  <= lSum:
  results = search.get_dict()

  for Result in results["local_results"]:
    lIdx += 1

    if lIdx <= lSum:

      #seleksi data (jangan ambil data jika jumlah review < 200 )
      if Result["reviews"] < 200:
        lIdx += 1
        continue
        
      # Append data sesuai dengan batas jumlah yang sudah ditentukan
      local_results.append({'Nama': Result["title"],
                           'data_id': Result["data_id"],
                           'total_reviews': Result["reviews"]});

  #menghilangkan pagination pada web, sehingga data yang terambil bisa banyak
  if "next" in results.get("serpapi_pagination", {}):
    # Akan mengubah parameter dari 'GoogleSearch()' dengan isi parameter dari halaman selanjutnya
    search.params_dict.update(dict(parse_qsl(urlsplit(results.get("serpapi_pagination").get("next")).query)))

  else:
    break


In [4]:
for LR in local_results:
  print(LR)
     

{'Nama': 'The Phoenix Hotel Yogyakarta - Handwritten Collection', 'data_id': '0x2e783ffff8aeead3:0xb9c77ab7663ae14d', 'total_reviews': 6226}
{'Nama': 'Aveta Hotel Malioboro', 'data_id': '0x2e7a59c6e514a5a3:0xbeca960436f8fe88', 'total_reviews': 2919}
{'Nama': 'Grand Zuri Malioboro Hotel', 'data_id': '0x2e7a582556020d3d:0x3ca3f23802afd105', 'total_reviews': 13411}
{'Nama': 'Malioboro Prime Hotel Yogyakarta', 'data_id': '0x2e7a5825e9ffaf5d:0xfe3ad08ccd87a1f2', 'total_reviews': 2572}
{'Nama': 'THE 1O1 Yogyakarta Tugu', 'data_id': '0x2e7a582ff893cce3:0x8bf559a1e440274', 'total_reviews': 11218}
{'Nama': 'Royal Malioboro by ASTON', 'data_id': '0x2e7a5824385c08fd:0xd21b79ff2f93808b', 'total_reviews': 1889}
{'Nama': 'Hotel NEO Malioboro', 'data_id': '0x2e7a5825c6f74639:0xe98585e928f8d15f', 'total_reviews': 10650}


In [5]:
data = {'Hotel_name':[],
        'name':[],
        'rating':[],
        'review':[]}

for LR in local_results:
  
  params = {
    "engine": "google_maps_reviews",
    "data_id": "",
    "api_key": "ed0a8d77bca60e3926ef66565842d540248b7614b12947115eb2e646258df5f8"
  }
  
  params["data_id"] = LR['data_id']

  search = GoogleSearch(params)

  lIdx = 0 # Angka indeks dari data
  lSum = 30 # Batas jumlah data yang ditampilkan

  while lIdx <= lSum:
    results = search.get_dict()

    for Result in results["reviews"]:
      lIdx += 1

      if lIdx <= lSum:
        # Append data sesuai dengan batas jumlah yang sudah ditentukan
        data['Hotel_name'].append(LR['Nama']);
        data['name'].append(Result["user"]["name"]);
        data['rating'].append(Result["rating"]);
        data['review'].append(Result["snippet"]);

    #menghilangkan pagination pada web, sehingga data yang terambil bisa banyak
    if "next" in results.get("serpapi_pagination", {}):
      # Akan mengubah parameter dari 'GoogleSearch()' dengan isi parameter dari halaman selanjutnya
      search.params_dict.update(dict(parse_qsl(urlsplit(results.get("serpapi_pagination").get("next")).query)))

    else:
      break
    
review_original = data['review']

In [6]:
import pandas as pd
dfScrap = pd.DataFrame(data)
dfScrap.to_csv(r'C:\Users\LENOVO\Desktop\Sentiment Analysis\reviewHotelJogja.csv', index=False)

In [7]:
dfScrap

,Hotel_name,name,rating,review
0,The Phoenix Hotel Yogyakarta - Handwritten Col...,Jim D,5.0,This is a fantastic hotel and a wonderful intr...
1,The Phoenix Hotel Yogyakarta - Handwritten Col...,Maja G,4.0,We spent 2 nights there with our 2 boys (age 1...
2,The Phoenix Hotel Yogyakarta - Handwritten Col...,Susan S,5.0,"Beautiful old hotel, with many special touches..."
3,The Phoenix Hotel Yogyakarta - Handwritten Col...,Jan-Markus H,4.0,If you like mix of colonial with Indonesian fl...
4,The Phoenix Hotel Yogyakarta - Handwritten Col...,Md Aman,3.0,Heritage hotel located near the Yogyakarta Mon...
...,...,...,...,...
205,Hotel NEO Malioboro,Darren W,5.0,The Orion room was spacious and the bed was co...
206,Hotel NEO Malioboro,wuryanti arman,5.0,"Their service is really great, very helpful. T..."
207,Hotel NEO Malioboro,Trip.com Member,3.7,"The cleanliness needs to be improved again, ye..."
208,Hotel NEO Malioboro,Trip.com Member,5.0,"Strategic..comfortable..can walk 24 hours, bus..."


Case Folding

In [8]:
import re
import string

In [9]:

review_clean = []

for review in review_original:
    # Remove Unicode
    review_test = re.sub(r'[^\x00-\x7F]+', ' ', review)
    # Remove Mentions
    review_test = re.sub(r'@\w+', '', review_test)
    # Lowercase the document
    review_test = review_test.lower()
    # Remove punctuations
    review_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', review_test)
    # Remove the numbers
    review_test = re.sub(r'[0-9]', '', review_test)
    # Remove the doubled space
    review_test = re.sub(r'\s{2,}', ' ', review_test)
    
    review_clean.append(review_test)

In [10]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
review_tokenized = []

for review in review_clean:
  review_tokenized.append(word_tokenize(review))

In [12]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
review_filtered = []
listStopword =  set(stopwords.words('english'))

for review in review_tokenized:
  
  review_each_filtered = []
  
  for r in review:
    if r not in listStopword:
        review_each_filtered.append(r)
        
  review_filtered.append(review_each_filtered)

In [14]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english")

review_stemmed = []
    
for review in review_filtered:
  
  review_each_stemmed = []
  
  for r in review:
    review_each_stemmed.append(stemmer.stem(r))
        
  review_stemmed.append(review_each_stemmed)

In [15]:
data_preprocessed = data
data_preprocessed['review'] = review_stemmed

df = pd.DataFrame(data_preprocessed)

df.to_csv(r'C:\Users\LENOVO\Desktop\Sentiment Analysis\reviewHotelJogjaPreprocessed.csv', index=False)

In [16]:
df

,Hotel_name,name,rating,review
0,The Phoenix Hotel Yogyakarta - Handwritten Col...,Jim D,5.0,"[fantast, hotel, wonder, introduct, indonesia,..."
1,The Phoenix Hotel Yogyakarta - Handwritten Col...,Maja G,4.0,"[spent, night, boy, age, staff, brilliant, ser..."
2,The Phoenix Hotel Yogyakarta - Handwritten Col...,Susan S,5.0,"[beauti, old, hotel, mani, special, touch, sui..."
3,The Phoenix Hotel Yogyakarta - Handwritten Col...,Jan-Markus H,4.0,"[like, mix, coloni, indonesian, flavor, hotel,..."
4,The Phoenix Hotel Yogyakarta - Handwritten Col...,Md Aman,3.0,"[heritag, hotel, locat, near, yogyakarta, monu..."
...,...,...,...,...
205,Hotel NEO Malioboro,Darren W,5.0,"[orion, room, spacious, bed, comfort, especi, ..."
206,Hotel NEO Malioboro,wuryanti arman,5.0,"[servic, realli, great, help, room, clean, com..."
207,Hotel NEO Malioboro,Trip.com Member,3.7,"[cleanli, need, improv, yesterday, stay, room,..."
208,Hotel NEO Malioboro,Trip.com Member,5.0,"[strateg, comfort, walk, hour, busi, near, cul..."
